In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import io
import os
import time
import re
import json
import sys
from tqdm.notebook import trange, tqdm #加進度條

In [3]:
URL =  "https://yoursalary.taiwanjobs.gov.tw/Salary/SalaryHome#"
chromedriver = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromedriver)

# # chrome_options.add_argument(‘–headless’)
# # chrome_options.add_argument(‘–disable-gpu’)
# # browser = webdriver.Chrome(options=chrome_options,executable_path=’chromedriver.exe’)

driver.get(URL)
now_handle = driver.current_window_handle
print(now_handle)

CDwindow-FC2E24F6892FFD8B98700ACFB172D3E5


In [81]:
# 先點圖片
driver.find_element_by_xpath("/html/body/div/div[2]/div[3]/div[1]/a/img").click()
time.sleep(5)
all_handles = driver.window_handles
for handle in all_handles:  
    if handle!=now_handle:     
            #輸出待選擇的視窗控制代碼  
            print(handle)
            driver.switch_to_window(handle)  
            time.sleep(1)

CDwindow-C5BB88009C2BF1CA19F85C5443BE9475


<ipython-input-81-54f882d843ab>:9: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(handle)


In [82]:
# 再點各縣市重要指標
element = driver.find_element_by_xpath("/html/body/form[1]/table/tbody/tr[2]/td/div/div[1]/table/tbody/tr[2]/td/table/tbody/tr/td/div/table/tbody/tr/td[3]/table/tbody/tr[4]/td[2]/table/tbody/tr/td/span/span")
element.click()
time.sleep(3)

In [119]:
# 建立list用來迴圈
date_lst = []
county_lst = []

date_elem = driver.find_element_by_xpath('/html/body/form[1]/table/tbody/tr[2]/td/div/div[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr/td[2]/div/table/tbody/tr/td[2]/div/select')
date = Select(date_elem)

county_elem = driver.find_element_by_xpath('/html/body/form[1]/table/tbody/tr[2]/td/div/div[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr/td[4]/div/table/tbody/tr/td[2]/div/select')
county = Select(county_elem)

for op in date.options:
    date_lst.append(op.text)

for oo in county.options:
    county_lst.append(oo.text)

date_lst = date_lst[2:]
county_lst.remove('六都')
print(date_lst)
print(county_lst)

['110年07月', '110年04月', '110年01月', '109年10月', '109年07月', '109年04月', '109年01月', '108年10月', '108年07月', '108年04月', '108年01月', '107年10月', '107年07月', '107年04月', '107年01月', '106年10月', '106年07月', '106年04月', '106年01月', '105年10月', '105年07月', '105年04月', '105年01月', '104年10月', '104年07月', '104年04月', '104年01月', '103年10月', '103年07月', '103年04月', '103年01月', '102年10月', '102年07月', '102年04月', '102年01月', '101年10月', '101年07月', '101年04月', '101年01月', '100年10月', '100年07月', '100年04月', '100年01月']
['全部', '新北市', '臺北市', '桃園市', '臺中市', '臺南市', '高雄市', '宜蘭縣', '新竹縣', '苗栗縣', '彰化縣', '南投縣', '雲林縣', '嘉義縣', '屏東縣', '臺東縣', '花蓮縣', '澎湖縣', '基隆市', '新竹市', '嘉義市', '金門縣', '連江縣']


In [149]:
DATE = []
COUNTY = []
WAGE = []
for i in date_lst:
    for j in county_lst:
        date = Select(driver.find_element_by_xpath('/html/body/form[1]/table/tbody/tr[2]/td/div/div[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr/td[2]/div/table/tbody/tr/td[2]/div/select'))
        county = Select(driver.find_element_by_xpath('/html/body/form[1]/table/tbody/tr[2]/td/div/div[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr/td[4]/div/table/tbody/tr/td[2]/div/select'))
        date.select_by_visible_text(i)
        county.select_by_visible_text(j)
        driver.find_element_by_xpath('/html/body/form[1]/table/tbody/tr[2]/td/div/div[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr/td[6]/div/button').click()
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        check = soup.find_all(name='table', attrs={'class':'xt'})
        if j != '全部':
            if len(check) > 2:
                table = check[2].find_all(name = 'tr')
                # 勞退全時工作平均提繳工資
                wage = table[6].getText()
                print("年份: %s, 縣市: %s, 工資: %s" % (i, j, wage))
            else:
                table = check[1].find_all(name = 'tr')
                # 勞退全時工作平均提繳工資
                wage = table[6].getText()
                print("年份: %s, 縣市: %s, 工資: %s" % (i, j, wage))
        else:
            table = soup.find_all(name='table', attrs={'class':'xt'})[0].find_all(name = 'tr')
            # 勞退全時工作平均提繳工資
            wage = table[6].getText().split(')')[1]
            print("年份: %s, 縣市: %s, 工資: %s" % (i, j, wage))
        DATE.append(i)
        COUNTY.append(j)
        WAGE.append(wage)

年份: 110年07月, 縣市: 全部, 工資: 43,506
年份: 110年07月, 縣市: 新北市, 工資: 40,647
年份: 110年07月, 縣市: 臺北市, 工資: 51,448
年份: 110年07月, 縣市: 桃園市, 工資: 43,092
年份: 110年07月, 縣市: 臺中市, 工資: 36,648
年份: 110年07月, 縣市: 臺南市, 工資: 37,139
年份: 110年07月, 縣市: 高雄市, 工資: 38,074
年份: 110年07月, 縣市: 宜蘭縣, 工資: 35,195
年份: 110年07月, 縣市: 新竹縣, 工資: 50,297
年份: 110年07月, 縣市: 苗栗縣, 工資: 39,778
年份: 110年07月, 縣市: 彰化縣, 工資: 33,578
年份: 110年07月, 縣市: 南投縣, 工資: 35,780
年份: 110年07月, 縣市: 雲林縣, 工資: 39,670
年份: 110年07月, 縣市: 嘉義縣, 工資: 35,302
年份: 110年07月, 縣市: 屏東縣, 工資: 33,588
年份: 110年07月, 縣市: 臺東縣, 工資: 33,477
年份: 110年07月, 縣市: 花蓮縣, 工資: 35,480
年份: 110年07月, 縣市: 澎湖縣, 工資: 34,792
年份: 110年07月, 縣市: 基隆市, 工資: 38,374
年份: 110年07月, 縣市: 新竹市, 工資: 66,028
年份: 110年07月, 縣市: 嘉義市, 工資: 35,303
年份: 110年07月, 縣市: 金門縣, 工資: 36,212
年份: 110年07月, 縣市: 連江縣, 工資: 38,072
年份: 110年04月, 縣市: 全部, 工資: 43,544
年份: 110年04月, 縣市: 新北市, 工資: 40,672
年份: 110年04月, 縣市: 臺北市, 工資: 51,548
年份: 110年04月, 縣市: 桃園市, 工資: 43,046
年份: 110年04月, 縣市: 臺中市, 工資: 36,675
年份: 110年04月, 縣市: 臺南市, 工資: 37,181
年份: 110年04月, 縣市: 高雄市, 工資: 38,278
年份: 110年04月,

年份: 108年01月, 縣市: 新竹市, 工資: 59,424
年份: 108年01月, 縣市: 嘉義市, 工資: 33,537
年份: 108年01月, 縣市: 金門縣, 工資: 35,436
年份: 108年01月, 縣市: 連江縣, 工資: 38,150
年份: 107年10月, 縣市: 全部, 工資: 40,808
年份: 107年10月, 縣市: 新北市, 工資: 38,525
年份: 107年10月, 縣市: 臺北市, 工資: 48,124
年份: 107年10月, 縣市: 桃園市, 工資: 40,827
年份: 107年10月, 縣市: 臺中市, 工資: 34,385
年份: 107年10月, 縣市: 臺南市, 工資: 34,992
年份: 107年10月, 縣市: 高雄市, 工資: 35,540
年份: 107年10月, 縣市: 宜蘭縣, 工資: 33,406
年份: 107年10月, 縣市: 新竹縣, 工資: 48,401
年份: 107年10月, 縣市: 苗栗縣, 工資: 37,901
年份: 107年10月, 縣市: 彰化縣, 工資: 31,647
年份: 107年10月, 縣市: 南投縣, 工資: 33,647
年份: 107年10月, 縣市: 雲林縣, 工資: 37,769
年份: 107年10月, 縣市: 嘉義縣, 工資: 33,303
年份: 107年10月, 縣市: 屏東縣, 工資: 31,404
年份: 107年10月, 縣市: 臺東縣, 工資: 32,013
年份: 107年10月, 縣市: 花蓮縣, 工資: 33,601
年份: 107年10月, 縣市: 澎湖縣, 工資: 33,520
年份: 107年10月, 縣市: 基隆市, 工資: 36,947
年份: 107年10月, 縣市: 新竹市, 工資: 59,152
年份: 107年10月, 縣市: 嘉義市, 工資: 33,024
年份: 107年10月, 縣市: 金門縣, 工資: 35,098
年份: 107年10月, 縣市: 連江縣, 工資: 37,874
年份: 107年07月, 縣市: 全部, 工資: 40,439
年份: 107年07月, 縣市: 新北市, 工資: 38,147
年份: 107年07月, 縣市: 臺北市, 工資: 47,778
年份: 107年07月,

年份: 105年04月, 縣市: 臺東縣, 工資: 30,188
年份: 105年04月, 縣市: 花蓮縣, 工資: 32,020
年份: 105年04月, 縣市: 澎湖縣, 工資: 31,032
年份: 105年04月, 縣市: 基隆市, 工資: 35,700
年份: 105年04月, 縣市: 新竹市, 工資: 54,369
年份: 105年04月, 縣市: 嘉義市, 工資: 31,250
年份: 105年04月, 縣市: 金門縣, 工資: 32,715
年份: 105年04月, 縣市: 連江縣, 工資: 34,104
年份: 105年01月, 縣市: 全部, 工資: 38,144
年份: 105年01月, 縣市: 新北市, 工資: 35,912
年份: 105年01月, 縣市: 臺北市, 工資: 45,071
年份: 105年01月, 縣市: 桃園市, 工資: 38,377
年份: 105年01月, 縣市: 臺中市, 工資: 31,640
年份: 105年01月, 縣市: 臺南市, 工資: 32,590
年份: 105年01月, 縣市: 高雄市, 工資: 32,894
年份: 105年01月, 縣市: 宜蘭縣, 工資: 31,165
年份: 105年01月, 縣市: 新竹縣, 工資: 48,184
年份: 105年01月, 縣市: 苗栗縣, 工資: 37,077
年份: 105年01月, 縣市: 彰化縣, 工資: 29,187
年份: 105年01月, 縣市: 南投縣, 工資: 31,176
年份: 105年01月, 縣市: 雲林縣, 工資: 35,220
年份: 105年01月, 縣市: 嘉義縣, 工資: 30,986
年份: 105年01月, 縣市: 屏東縣, 工資: 29,409
年份: 105年01月, 縣市: 臺東縣, 工資: 30,017
年份: 105年01月, 縣市: 花蓮縣, 工資: 31,739
年份: 105年01月, 縣市: 澎湖縣, 工資: 31,007
年份: 105年01月, 縣市: 基隆市, 工資: 35,542
年份: 105年01月, 縣市: 新竹市, 工資: 54,379
年份: 105年01月, 縣市: 嘉義市, 工資: 30,927
年份: 105年01月, 縣市: 金門縣, 工資: 32,414
年份: 105年01月

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=93.0.4577.82)


In [150]:
c={"Date" : DATE,
   "County" : COUNTY,
   "Wage" : WAGE}
df = pd.DataFrame(c)

In [151]:
df

,Date,County,Wage
0,110年07月,全部,"43,506"
1,110年07月,新北市,"40,647"
2,110年07月,臺北市,"51,448"
3,110年07月,桃園市,"43,092"
4,110年07月,臺中市,"36,648"
...,...,...,...
692,103年01月,臺北市,"41,581"
693,103年01月,桃園市,"35,972"
694,103年01月,臺中市,"29,220"
695,103年01月,臺南市,"30,195"


In [152]:
# 存成excel
df.to_excel('./工資.xlsx', index = False)